In [14]:
import json
import os

from datasets import load_dataset, concatenate_datasets

from config import load_config
from util.parsing import get_features
from util.windowing import make_random_window

## Get and Load Config and Datasets

In [15]:
configs = []
cfg = load_config(configs)
with open(cfg['mappings'], 'r') as fp:
    phoneme_mappings = json.load(fp)

In [16]:
rd = load_dataset(cfg['datasets']['russian'])
pd = load_dataset(cfg['datasets']['polish'])

## Generate Windows

In [ ]:
def preprocess(example):
    sentence = example['text']
    phoneme_sentence = example['text-phoneme']

    window_settings = cfg['windows']
    min_window_length = window_settings['min_size']
    max_window_length = window_settings['max_size']
    window_length_decay = window_settings['size_decay']

    """
    TODO
    because the phonemizer is word-level, you can use spaces to create boundaries for your windows
    by doing that you can create windows for both the phonetic and the orthographic columns.

    I'll leave it up to you as to how you want to encode the data into the dataset, but I'll tell you we need:
    1. The window start and stop characters
    2. The phonological features present in the span
    3. You need a window for both the orthographic column and the phonetic column
        * If you want to use the same window that's fine, but then the window offsets need to be word-based instead of character based.
        * You can see my implementation of the question answering datasets to see how you can use dicts in dataset columns: https://huggingface.co/datasets/iggy12345/sberquad-ipa (specifically see `answers` and `answers-phoneme`)
    """

    result = {}

    # TODO write your code here

    return result


keep_columns = ['text', 'text-phoneme']
rd_sampled = rd.map(preprocess, remove_columns=[c for c in rd.column_names if c not in keep_columns], num_proc=os.cpu_count())
pd_sampled = pd.map(preprocess, remove_columns=[c for c in rd.column_names if c not in keep_columns], num_proc=os.cpu_count())

# Concatenate Datasets

In [ ]:
combined_ds = concatenate_datasets([rd_sampled, pd_sampled])

## Push to Huggingface

In [ ]:
# TODO if you get here, you're done! I'll handle pushing the dataset to the cloud

target_dataset_name = 'rus-pol-edge-probing-phono-feats'
username = 'iggy12345'

In [ ]:
combined_ds.push_to_hub(f'{username}/{target_dataset_name}')